In [ ]:
!git clone https://github.com/Toloka/VLDB2021_Crowd_Science_Challenge.git

Cloning into 'VLDB2021_Crowd_Science_Challenge'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 131 (delta 63), reused 26 (delta 10), pack-reused 0
Receiving objects: 100% (131/131), 2.03 MiB | 6.53 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [ ]:
%cd VLDB2021_Crowd_Science_Challenge/

/content/VLDB2021_Crowd_Science_Challenge


In [ ]:
import pandas as pd
import numpy as np
import argparse
from tqdm.auto import tqdm

SEED = 42

from collections import Counter
  
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]
    
def filter_text(text):
    char_set = set(list(' qwertyuiopasdfghjklzxcvbnm\''))
    return ''.join([c for c in text.strip().lower() if c in char_set])

In [ ]:
!pip install jiwer

     |████████████████████████████████| 51kB 5.7MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149828 sha256=635f02aaf2cede99f1d717e2dc16e840c4fbc6cb172d94722505bb6e06ed9531
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers
!pip install -e .

Cloning into 'transformers'...
remote: Enumerating objects: 74620, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 74620 (delta 19), reused 23 (delta 13), pack-reused 74583
Receiving objects: 100% (74620/74620), 57.26 MiB | 8.55 MiB/s, done.
Resolving deltas: 100% (53161/53161), done.
/content/transformers
Obtaining file:///content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 901kB 14.3MB/s 
     |████████████████████████████████| 3.3MB 28.5MB/s 
  Running setup.py develop for transformers


In [ ]:
%%capture
#!pip install transformers
!pip install -r /content/transformers/examples/pytorch/summarization/requirements.txt
!pip install wandb -qq

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
%env WANDB_WATCH=all
%env WANDB_PROJECT=yandex-T5-large_all
%env TASK_NAME=yandex-T5-large_all

env: WANDB_WATCH=all
env: WANDB_PROJECT=yandex-T5-large_all
env: TASK_NAME=yandex-T5-large_all


In [ ]:
import pandas as pd
import jiwer

In [ ]:
data_1 = pd.read_csv(r'/data/main_train.csv', index_col=0)
data_2 = pd.read_csv(r'/data/TTS_to_STT.csv', index_col=0)
data_3 = pd.read_csv(r'/data/twiiter_dataset.csv', index_col=0)
data_4 = pd.read_csv(r'/data/dsts2_dataset.csv', index_col=0).dropna()


data_cat = pd.concat([data_1, 
                      data_2])

In [ ]:
#data_1 = pd.read_csv(r'/content/main_train.csv', index_col=0)
#data_2 = pd.read_csv(r'/content/TTS_to_STT.csv', index_col=0)
#data_3 = pd.read_csv(r'/content/twiiter_dataset.csv', index_col=0)
#data_4 = pd.read_csv(r'/content/dsts2_dataset.csv', index_col=0).dropna()

#data_cat = pd.concat([data_1, data_2, data_3, data_4])

In [ ]:
def filter_text(text):
    char_set = set(list(' qwertyuiopasdfghjklzxcvbnm\''))
    text = text.strip("''")
    text = text.replace(" '","'")
    return ''.join([c for c in text.strip().lower() if c in char_set])


transformation = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemoveMultipleSpaces(),
    jiwer.RemoveWhiteSpace(replace_by_space=True),
    #jiwer.RemovePunctuation(),
    jiwer.Strip(),
    #jiwer.SentencesToListOfWords(word_delimiter=" ")
]) 

In [ ]:
data_cat = data_cat[['output','task']].applymap(filter_text).applymap(transformation)
data_cat

,output,task
0,he's doctor recommended that he take walks to ...,his doctor recommended that he take walks to r...
1,his doctor recommended that he take walks to r...,his doctor recommended that he take walks to r...
2,his doctor recommended that he takes walks to ...,his doctor recommended that he take walks to r...
3,his doctor recommended that he take walks to r...,his doctor recommended that he take walks to r...
4,his doctor recommended that he take walks to r...,his doctor recommended that he take walks to r...
...,...,...
95,how can i get from guards into murray and plots,how can i get from garching to marienplatz
96,when does the next train leaves from quench tr...,when does the next train leaves from quiddestrae
97,how can i get is handling a chore,how can i get to sendlinger tor
98,when in the next train leaving in gardening,when is the next train leaving in garching


In [ ]:
data_cat = data_cat[data_cat.output.str.len() > 5]
data_cat = data_cat[data_cat.task.str.len() > 5]

In [ ]:
from jiwer import wer
data_wer = data_cat.copy()
data_wer['WER'] = data_wer.apply(lambda a: 1-wer(a.task, a.output), axis=1)

data_cat = data_cat[data_cat.output.str.len() > 5]
data_cat = data_cat[data_cat.task.str.len() > 5]

#68232 rows × 3 columns
data_wer

,output,task,WER
0,he's doctor recommended that he take walks to ...,his doctor recommended that he take walks to r...,0.909091
1,his doctor recommended that he take walks to r...,his doctor recommended that he take walks to r...,1.000000
2,his doctor recommended that he takes walks to ...,his doctor recommended that he take walks to r...,0.909091
3,his doctor recommended that he take walks to r...,his doctor recommended that he take walks to r...,0.818182
4,his doctor recommended that he take walks to r...,his doctor recommended that he take walks to r...,0.909091
...,...,...,...
95,how can i get from guards into murray and plots,how can i get from garching to marienplatz,0.375000
96,when does the next train leaves from quench tr...,when does the next train leaves from quiddestrae,0.625000
97,how can i get is handling a chore,how can i get to sendlinger tor,0.428571
98,when in the next train leaving in gardening,when is the next train leaving in garching,0.750000


In [ ]:
data_wer

,output,task,WER
0,he's doctor recommended that he take walks to ...,his doctor recommended that he take walks to r...,0.909091
1,his doctor recommended that he take walks to r...,his doctor recommended that he take walks to r...,1.000000
2,his doctor recommended that he takes walks to ...,his doctor recommended that he take walks to r...,0.909091
3,his doctor recommended that he take walks to r...,his doctor recommended that he take walks to r...,0.818182
4,his doctor recommended that he take walks to r...,his doctor recommended that he take walks to r...,0.909091
...,...,...,...
95,how can i get from guards into murray and plots,how can i get from garching to marienplatz,0.375000
96,when does the next train leaves from quench tr...,when does the next train leaves from quiddestrae,0.625000
97,how can i get is handling a chore,how can i get to sendlinger tor,0.428571
98,when in the next train leaving in gardening,when is the next train leaving in garching,0.750000


In [ ]:
data_group = data_wer[data_wer.WER > 0.0].groupby('task').agg(list)
data_group = data_group.reset_index()
data_group.output = data_group.output.apply(lambda a: ' | '.join(a))

In [ ]:
data_group

,task,output,WER
0,a beautiful light surrounds the mask drawing i...,it gives full light surrounds the mass prior t...,"[0.16666666666666663, 0.9166666666666666, 0.83..."
1,a blacksmith courted me nine months and better...,a blacksmith courted me nine months and better...,"[1.0, 0.8235294117647058, 1.0, 0.7647058823529..."
2,a blank cell indicates no change,a blank cell indicates no change. your blank s...,"[1.0, 0.6666666666666667, 0.33333333333333337,..."
3,a bone spavin case study,it bones kivn study. a bones fave in case stud...,"[0.19999999999999996, 0.4, 0.4, 0.4, 0.4]"
4,a bottle of gin and an automatic pistol were f...,a bottle of chin and an automatic pestal were ...,"[0.875, 0.875, 1.0, 0.5625, 0.75, 0.875, 1.0]"
...,...,...,...
9791,your mother won't mind you giving this to me,your mother won't mind you giving this to me. ...,"[1.0, 0.4444444444444444, 0.2222222222222222, ..."
9792,your number's up and it is missing,your number's up and it is missing. your numbe...,"[1.0, 0.5714285714285714, 0.7142857142857143, ..."
9793,your price just wasn't high enough,you price just wasn't high enough. your price ...,"[0.8333333333333334, 1.0, 0.6666666666666667, ..."
9794,your sustained commitment and your willingness...,your sustained commitment and your willingness...,"[0.9285714285714286, 0.9285714285714286, 0.785..."


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_group, test_size=0.1, random_state=SEED)
train.to_csv('/content/train_.csv')
test.to_csv('/content/test_.csv')

In [ ]:
#!rm -r /tmp/t5-large-6_11

In [ ]:
#new-simple
!python /content/transformers/examples/pytorch/summarization/run_summarization.py \
    --seed 42 \
    --model_name_or_path t5-large \
    --num_train_epochs 8 \
    --warmup_ratio 0.1 \
    --do_train \
    --do_eval \
    --train_file /content/train_.csv \
    --validation_file /content/test_.csv \
    --source_prefix "summarize: " \
    --output_dir /tmp/t5-large/ \
    --overwrite_output_dir \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 6 \
    --text_column 'output' \
    --summary_column 'task' \
    --save_total_limit 8 \
    --gradient_accumulation_steps 1 \
    --evaluation_strategy "steps" \
    --eval_steps 1000 \
    --logging_steps 1000 \
    --save_steps 1000 \
    --predict_with_generate \
        #--do_eval \
#            --max_source_length 256 \
#    --max_target_length 64 \

2021-06-12 10:36:26.145840: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
06/12/2021 10:36:27 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
06/12/2021 10:36:27 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=1000,
evaluation_strategy=IntervalStrategy.STEPS,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5

In [ ]:
data_eval = pd.read_csv(r'/content/VLDB2021_Crowd_Science_Challenge/data/test/responses.csv', index_col=0)
data_eval = data_eval[['output']].applymap(filter_text).applymap(transformation)
data_eval = data_eval.groupby('task').agg(list)
data_eval.output = data_eval.output.apply(lambda a: ' | '.join(a))
data_eval = data_eval.reset_index()
data_eval['task'] = 'a'
data_eval
data_eval.to_csv(r'/content/data_eval_.csv')

In [ ]:
!python /content/transformers/examples/pytorch/summarization/run_summarization.py \
    --model_name_or_path /tmp/t5-large \
    --do_predict \
    --validation_file /content/data_eval_.csv \
    --test_file /content/data_eval_.csv \
    --source_prefix "summarize: " \
    --output_dir /content/eval_2_ \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --predict_with_generate \
    --text_column 'output' \
    --summary_column 'task' \
    --num_beams 12 \
#    --num_beam_groups 4 \
    #    --min_length 10 \

2021-06-12 13:19:24.302957: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
06/12/2021 13:19:25 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
06/12/2021 13:19:25 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=True,
do_train=False,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-0

In [ ]:
import pandas as pd

mass_ = []
with open(r'/content/eval_2_/generated_predictions.txt','r') as fp:
    for ii in fp.readlines():
        mass_.append(ii.split(" | ")[0])

ddf = pd.DataFrame()
ddf['output'] = mass_
#ddf.
ddf.output.replace(r'\n','', regex=True).to_csv('t5-large_ans.csv')